# Nye kanon for lærerutdanningen - februar 2020

Nedenfor går vi gjennom en notebook for å studere et mindre korpus av bøker (under 100), finne ut noe av det tematiske, hvordan ord henger sammen gjennom korpuset, samt bruk av metadata og lesbarhetsindeks (TTR).

Først henter vi inn det som trengs for at notebooken skal virke.

In [1]:
import dhlab.nbtext as nb
from dhlab.module_update import css, code_toggle
import pandas as pd
import qgrid as qg
css()

### Her defineres noen hjelpekommandoer for kanon-prosjektet

Det er ikke nødvendig å forstå definisjonen av kommandoene, men greit å vite hva de gjør. Hopp over i første omgang, og sjekk etter å ha sett hvordan kommandoen brukes.

#### hjelpekommando for å sjekke ord


In [2]:
sjekk_ord = lambda x: texts.loc[x].dropna()

#### en kommando som viser en dataramme med tall som et varmekart - fin å bruke for dokument-term-matriser.


In [3]:
varmekart = lambda t, top=5: t.head(top).fillna(0).style.background_gradient()

#### Kommando for å se på ord gjennom hele korpuset

Variabelen `dtm` er dokument-term-matrisen, og `ordliste` er en liste med ord skilt med blanke tegn.

In [4]:
def oversikt(dtm, ordliste):
    if isinstance(ordliste, str):
        ordliste = ordliste.split() 
    df = dtm.reindex(ordliste).loc[ordliste].transpose().fillna(0)
    df.index = bok_info(nb.metadata(list(df.index)))
    return df.style.background_gradient()

#### bok_info smelter sammen informasjon om metadata



In [5]:
bok_info = lambda b: ['-'.join([str(y) for y in x[0:4]]) for x in b]

#### kommandoen `bøker` tar et `søkeord` og viser en liste over de bøker ordet forekommer i, sortert på frekvens.



In [6]:
def bøker(søkeord):
    try:
        data = sjekk_ord(søkeord)
        df = nb.frame(nb.metadata(list(data.index)), "urn forfatter år tittel subtittel forlag sjanger".split())["urn forfatter år tittel subtittel forlag sjanger".split()]
        df[søkeord] = data.values
        df = df.sort_values(by=søkeord, ascending = False)
    except:
        df = pd.DataFrame()
    return df

## Lag et korpus

Korpus bygges gjennom å velge bøker fra bokhylla. Alle bøker har en såkalt URN (Universal Resource Name) som entydig peker på den digitale kopien. Den består av en serie bilder av boksidene, i tillegg til den OCR-behandlede teksten. Det er teksten som gjøres gjenstand for analyse.

Et korpus velges gjennom kommandoen `book_corpus`, som tar et sett parametre for å begrense utvalget, som periode, dewey, emneord, forfatter, tittel og språk, med mer.

In [7]:
korpus = nb.book_corpus(subject = 'ungdom', period = (1980, 2000), lang='nob', limit = 50 )

In [8]:
korpus

,urn,author,title,year
0,2014061806198,"Bjerka, Johan",Ungdom og satanisme,1993
1,2009031700084,"Skretting, Astrid",Ungdom og rusmidler,2000
2,2008061004029,"Seim, Sol",Tenåringen blir voksen og eldre,1988
3,2007101100004,"Heinemann, Peter Paul",Skulke livet,1991
4,2009010700051,"Olweus, Dan",Mobbing blant barn og unge,1997
5,2011020308079,,Forskningsprogram blir til,1995
6,2011011808092,"Fauske, Halvor",Hvordan skal framtida formes?,1991
7,2010061008025,"Aagre, Willy",Ressursbok for læreren,1996
8,2012071308086,"Øvrebø, Else Marie",Kostvaner blant 14-åringer i Murmansk og Tromsø,2000
9,2012062606024,"Werner, Anita",Ungdom og video,1982


# Behandling

Korpuset kan analyseres med hensyn til tema og lesbarhet, i tillegg til annen statistikk. Det kan også søkes i, for å se litt om kontekster og hva som behandles. Konkordansene kan også gi et innblikk i hva som tematisk behandles.

In [9]:
nb.urn_concordance(word='forelsket', urns = nb.pure_urn(korpus), before = 10, after = 10)

"Skulke livet, Heinemann, Peter Paul, 1991",ble vennlig mottatt . Jeg trodde at hun kanskje hadde,forelsket,"seg i en innvandrer , og var egentlig veldig lettet"
"Skulke livet, Heinemann, Peter Paul, 1991",innledet straks et forhold til henne . Hun var veldig,forelsket,"i ham , og ble raskt avhengig av de samme"
"samfunnskritiske ungdomsromanen, Granberg, Mia, 1982","fredagen går han hjem til Janne , som han er",forelsket,i . Janne skal passe lillebroren mens foreldrene er ute
"samfunnskritiske ungdomsromanen, Granberg, Mia, 1982","til foreldrene , vanker sammen med gjengen , og er",forelsket,"i Helge . Helge er nylig flyttet til strøket ,"
"Helsefremmende og forebyggende arbeid for barn og unge 0-20 år i helsestasjons- og skolehelsetjenesten, , 1998",grunn til at de har sex er at de er,forelsket,. Samtidig er de yngste ( 15 - 16 år
"Moderne svartedauen?, , 1988",", det var jo tross alt ikke meg han var",forelsket,i . Tanken om AIDS-fare streifet meg ikke på det
"Treffpunkt Unge røster, Vilsvik, Astrid, 1997",Strever med å bli sikker på at den du er,forelsket,"i , er den rette . Gi deg tid !"
"Kjærlighetsartikler, Rudberg, Monica, 1997",de mannlige hovedpersonene at han kunne tenke seg å bli,forelsket,- noe han i løpet av sine 30 leveår aldri
"Kjærlighetsartikler, Rudberg, Monica, 1997",mannlige hovedpersonen sier at han kunne tenke seg å bli,forelsket,"( noe han aldri har vært ) , bare for"
"Kjærlighetsartikler, Rudberg, Monica, 1997",Hammelow-Berg : « Er du en type som blir fort,forelsket,? » Bjørn : « Nei - men jeg må
"Kjærlighetsartikler, Rudberg, Monica, 1997",at jeg har vorti det her . » « Skikkelig,forelsket,? » Nikk ( jubel ) .


# Hent tekstene som frekvenslister over ord

Tekstene kan lastes ned som en såkalt dokument-term-matrise. Den gjør det mulig å se på hvordan ord fordeler gjennom korpuset.

In [10]:
texts = nb.get_corpus_text(nb.pure_urn(korpus), cutoff = 1)

### Slik ser tekstene ut som en dokument-term-matrise

Her viser vi tekstene med et varmekart som gir et raskt overblikk over forskjeller mellom dem.

In [11]:
varmekart(texts)

,2014061806198,2009031700084,2008061004029,2007101100004,2009010700051,2011020308079,2011011808092,2010061008025,2012071308086,2012062606024,2009072101037,2011022805021,2015010808073,2008090200001,2007110901012,2007071101045,2008020501059,2014031706053,2010052106017,2013012506172,2010021904014,2007112701005,2008053004037,2008060504092,2012080108086,2011011405038,2012092508018,2016030709052,2009030900078,2010041903044,2013100106095,2008081304080,2008081204030,2009030404117,2009082400030,2009120104138,2009083100053,2012121306016,2008090304027,2010011400003,2009093000117,2010113008139,2015012708028,2014070905004,2009093000016,2015070209019,2012072409127,2007110104033,2015090706125,2009062900032
.,1871,770,1187,4361,431,3842,745,889,1552,605,889,3302,559,6849,4127,2320,183,4757,1186,1191,3285,6199,1096,2289,279,1067,1119,1112,241,1143,4804,3110,2832,1035,860,2121,942,651,3912,3546,1262,1017,1748,441,4319,640,531,693,2392,1394
",",1242,750,929,3047,318,3337,415,570,1033,355,515,1819,255,3874,2848,1568,153,3042,436,428,2116,7282,642,1719,89,709,406,514,249,674,3165,1960,3107,628,839,2109,787,399,3639,1688,925,269,944,304,2507,367,294,337,188,1261
som,815,561,413,1619,149,926,313,371,430,148,312,798,221,2487,1458,804,79,1492,442,340,689,2017,485,652,52,297,257,404,94,83,1999,623,2352,553,126,947,455,47,1521,1159,938,389,280,92,1355,224,167,159,49,691
i,815,831,649,1509,208,1016,389,611,1305,493,543,910,352,4696,1799,1103,140,1229,541,762,1453,2442,788,728,72,429,642,462,138,629,2212,1097,2015,672,325,1043,554,114,1838,1641,884,325,480,195,2123,386,301,278,175,949
og,716,598,653,3069,292,1409,467,812,790,350,459,1178,346,5333,2348,1686,172,2580,1059,808,2790,4078,499,1089,56,478,772,475,189,850,4202,1715,2093,663,451,1576,479,29,2631,2189,659,332,717,174,2176,245,310,242,306,1127


## Spredning av ord

Forekomster av et vilkårlig sett med ord kan visualiseres gjennom korpuset. Varmekartet gir et godt blikk på hvordan ordene fordeler seg og samvarierer.

In [12]:
oversikt(texts, ['forelsket', 'vennskap', 'gutter', 'jenter','miljø', 'skog', 'fotball'])

,forelsket,vennskap,gutter,jenter,miljø,skog,fotball
"2014061806198-Bjerka, Johan-1993-Ungdom og satanisme",0,5,6,0,2,0,0
"2009031700084-Skretting, Astrid-2000-Ungdom og rusmidler",0,0,96,97,0,0,0
"2008061004029-Seim, Sol-1988-Tenåringen blir voksen og eldre",0,0,6,0,6,0,0
"2007101100004-Heinemann, Peter Paul-1991-Skulke livet",2,0,2,2,7,0,0
"2009010700051-Olweus, Dan-1997-Mobbing blant barn og unge",0,0,15,5,0,0,0
2011020308079--1995-Forskningsprogram blir til,0,0,16,17,37,6,0
"2011011808092-Fauske, Halvor-1991-Hvordan skal framtida formes?",0,0,4,9,2,0,0
"2010061008025-Aagre, Willy-1996-Ressursbok for læreren",0,5,14,14,0,0,0
"2012071308086-Øvrebø, Else Marie-2000-Kostvaner blant 14-åringer i Murmansk og Tromsø",0,0,38,43,7,0,0
"2012062606024-Werner, Anita-1982-Ungdom og video",0,0,10,12,0,0,0


# Metadata

Over lagde vi en hjelpekommando, kalt `bøker`, som tar et ord som parameter, og lister ut alle bøkene som har ordet, sortert på frekvensen. Den gir litt mer detalj om metadata enn kommandoen `oversikt`.

In [13]:
bøker('miljø')

,urn,forfatter,år,tittel,subtittel,forlag,sjanger,miljø
20,2013100106095,"Larsen, Erik",1983,Fra avvik til ansvar,en miljøterapeutisk tilnærming til ungdom i in...,Tanum-Norli,notfiction,55.0
26,2008090304027,"Frønes, Ivar",1998,De likeverdige,om sosialisering og de jevnaldrendes betydning,Universitetsforl.,notfiction,48.0
3,2011020308079,,1995,Forskningsprogram blir til,rapport fra det første programseminaret om mil...,Norges forskningsråd,unknown,37.0
14,2010021904014,,1998,Helsefremmende og forebyggende arbeid for barn...,,Statens helsetilsyn,unknown,36.0
15,2007112701005,"Tetzchner, Stephen von",1992,Habilitering,tverrfaglig arbeid for mennesker med medfødte ...,Ad Notam Gyldendal,notfiction,26.0
9,2007071101045,"Granberg, Mia",1982,samfunnskritiske ungdomsromanen,,Landslaget for norskundervisning : Cappelen,notfiction,22.0
7,2008090200001,,1999,Ungdomstiltak i større bysamfunn,en evaluering,"Norsk institutt for forskning om oppvekst, vel...",unknown,21.0
28,2009093000016,"Strandbu, Åse",2000,Storbyungdom og natur,en undersøkelse av Osloungdoms forhold til fri...,"Norsk institutt for forskning om oppvekst, vel...",unknown,18.0
13,2013012506172,"Waal, Helge",1984,Grunnskolen og den vanskelige elev,en vurdering av skolesystemets plass og muligh...,Det Kgl. sosialdepartement;Oslo : Det Kgl. sos...,notfiction,14.0
6,2011022805021,"Varang, Kjersti",2000,På kant med loven og seg selv,to guttegjenger i en bydel i Oslo,Det kriminalitetsforebyggende råd,notfiction,13.0


In [14]:
bøker('fotball')

,urn,forfatter,år,tittel,subtittel,forlag,sjanger,fotball
0,2008090200001,,1999,Ungdomstiltak i større bysamfunn,en evaluering,"Norsk institutt for forskning om oppvekst, vel...",unknown,6.0
9,2007110104033,"Naguib, Nefissa",1991,delt liv,en rapport om unge flyktninger i Norge,Redd Barna,unknown,5.0
1,2014031706053,"Ladberg, Gunilla",1980,Utviklingspsykologi,,Gyldendal,notfiction,4.0
3,2008053004037,"Grue, Lars",1982,organiserte ungdommen,en empirisk analyse :sluttrapport fra fase én ...,[s.n.],notfiction,3.0
4,2013100106095,"Larsen, Erik",1983,Fra avvik til ansvar,en miljøterapeutisk tilnærming til ungdom i in...,Tanum-Norli,notfiction,3.0
7,2008090304027,"Frønes, Ivar",1998,De likeverdige,om sosialisering og de jevnaldrendes betydning,Universitetsforl.,notfiction,3.0
8,2009093000016,"Strandbu, Åse",2000,Storbyungdom og natur,en undersøkelse av Osloungdoms forhold til fri...,"Norsk institutt for forskning om oppvekst, vel...",unknown,3.0
2,2007112701005,"Tetzchner, Stephen von",1992,Habilitering,tverrfaglig arbeid for mennesker med medfødte ...,Ad Notam Gyldendal,notfiction,2.0
5,2008081204030,"Rudberg, Monica",1997,Kjærlighetsartikler,"ungdom, kjønn og kjærlighet i forandring",Tano Aschehoug,notfiction,2.0
6,2009120104138,"Gillberg, Christopher",1998,Ett barn i hver klasse,om barn og unge med DAMP/MBD og ADHD,Praxis forl.,notfiction,2.0


# Lesbarhetsindekser

Alle bøkene kan beskrives gjennom en lesbarhetsindeks, standarden er STTR (Standardized Type Toke Ratio). Med standarized menes at indeksen er rimelig rettferdig uavhengig av bokens lengde. Her er tallet et gjennomsnitt av å dele inn hver bok i seksjoner på 5000 ord hver.

### Bygg lesbarhetsindeksen

Indeksen bygges med kommandoen `sttr`.

In [15]:
lesbarhet = nb.frame({b:nb.sttr(b) for b in korpus.urn}, name='indeks')

### Henter ut metada for bøkene for orientering

Det følgende er mest Python-kode, som ikke er essensiell for forståelsen av det som skjer, men kan stå som en dokumentasjon på det som er gjort for å lage sluttproduktet vist fire celler under.

In [16]:
bøker_meta = nb.metadata(list(lesbarhet.index))

In [17]:
lesbarhet['info'] = bok_info(bøker_meta)

### Resultatet av lesbarhetsindeksen

In [18]:
nb.frame_sort(lesbarhet, by='indeks')

,indeks,info
2015090706125,0.497836,2015090706125--1987-Våre bøker
2009082400030,0.422937,2009082400030--1993-Norske allaktivitetshus
2016030709052,0.419304,"2016030709052-Grimshei, Christian-1998-Om hous..."
2014070905004,0.418158,"2014070905004-Stangeland, Per-1981-Evaluering ..."
2009010700051,0.410159,"2009010700051-Olweus, Dan-1997-Mobbing blant b..."
2011020308079,0.388774,2011020308079--1995-Forskningsprogram blir til
2011011808092,0.376120,"2011011808092-Fauske, Halvor-1991-Hvordan skal..."
2011011405038,0.370219,"2011011405038-Roe, Jan-Erik Bernes-1993-Spisef..."
2010021904014,0.360521,2010021904014--1998-Helsefremmende og forebygg...
2014061806198,0.356341,"2014061806198-Bjerka, Johan-1993-Ungdom og sat..."
